In [11]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML

test = """PokerStars Hand #242207161368:  Fusion Pot Limit (€0.01/€0.02 EUR) - 2023/03/03 20:38:34 CET [2023/03/03 14:38:34 ET]
Table 'Baetsle III' 6-max Seat #3 is the button
Seat 2: jeje_sat (€2 in chips) 
Seat 3: MERGUEZ-3510 (€2.98 in chips) 
Seat 5: joconde421 (€1.85 in chips) 
joconde421: posts small blind €0.01
jeje_sat: posts big blind €0.02
*** HOLE CARDS ***
Dealt to jeje_sat [6h Jh]
MERGUEZ-3510: folds 
joconde421: calls €0.01
jopatate07 leaves the table
jeje_sat: checks 
*** FLOP *** [Qs 9c Jd]
Dealt to jeje_sat [6h Jh] [5c]
joconde421: checks 
jeje_sat: checks 
*** TURN *** [Qs 9c Jd] [4c]
Dealt to jeje_sat [6h Jh 5c] [3d]
joconde421: checks 
jeje_sat: checks 
*** RIVER *** [Qs 9c Jd 4c] [2c]
joconde421: checks 
Amir2800 joins the table at seat #4 
jeje_sat: checks 
*** SHOW DOWN ***
joconde421: shows [Qc 8h Ah 6s] (a pair of Queens)
jeje_sat: mucks hand 
joconde421 collected €0.04 from pot
*** SUMMARY ***
Total pot €0.04 | Rake €0 
Board [Qs 9c Jd 4c 2c]
Seat 2: jeje_sat (big blind) mucked [6h Jh 5c 3d]
Seat 3: MERGUEZ-3510 (button) folded before Flop (didn't bet)
Seat 5: joconde421 (small blind) showed [Qc 8h Ah 6s] and won (€0.04) with a pair of Queens"""
    

substitutions = {
                     'LEGAL_ISO' : "USD|EUR|GBP|CAD|FPP|SC|INR|CNY",      # legal ISO currency codes
                            'LS' : u"\$|\xe2\x82\xac|\u20ac|\£|\u20b9|\¥|", # legal currency symbols - Euro(cp1252, utf-8)
                           'PLYR': r'\s?(?P<PNAME>.+?)',
                            'CUR': u"(\$|\xe2\x82\xac|\u20ac|€|\£|\u20b9|\¥|)",
                          'BRKTS': r'(\(button\) |\(small blind\) |\(big blind\) |\(button blind\) |\(button\) \(small blind\) |\(small blind\) \(button\) |\(big blind\) \(button\) |\(small blind/button\) |\(button\) \(big blind\) )?',
                    }

re_GameInfo = re.compile(u"""(?P<SITE>PokerStars|POKERSTARS|Hive\sPoker|Full\sTilt|PokerMaster|Run\sIt\sOnce\sPoker|BetOnline|PokerBros)(?P<TITLE>\sGame|\sHand|\sHome\sGame|\sHome\sGame\sHand|Game|\s(Zoom|Rush)\sHand|\sGAME)\s\#(?P<HID>[0-9]+):\s+
          (\{.*\}\s+)?((?P<TOUR>((Zoom|Rush)\s)?(Tournament|TOURNAMENT))\s\#                # open paren of tournament info
          (?P<TOURNO>\d+),\s(Table\s\#(?P<HIVETABLE>\d+),\s)?
          # here's how I plan to use LS
          (?P<BUYIN>(?P<BIAMT>[%(LS)s\d\.]+)?\+?(?P<BIRAKE>[%(LS)s\d\.]+)?\+?(?P<BOUNTY>[%(LS)s\d\.]+)?\s?(?P<TOUR_ISO>%(LEGAL_ISO)s)?|Freeroll|)(\s+)?(-\s)?
          (\s.+?,)?
          )?
          # close paren of tournament info
          (?P<MIXED>HORSE|8\-Game|8\-GAME|HOSE|Mixed\sOmaha\sH/L|Mixed\sHold\'em|Mixed\sPLH/PLO|Mixed\sNLH/PLO|Mixed\sOmaha|Triple\sStud)?\s?\(?
          (?P<SPLIT>Split)?\s?
          (?P<GAME>Hold\'em|HOLD\'EM|Hold\'em|6\+\sHold\'em|Razz|RAZZ|Fusion|7\sCard\sStud|7\sCARD\sSTUD|7\sCard\sStud\sHi/Lo|7\sCARD\sSTUD\sHI/LO|Omaha|OMAHA|Omaha\sHi/Lo|OMAHA\sHI/LO|Badugi|Triple\sDraw\s2\-7\sLowball|Single\sDraw\s2\-7\sLowball|5\sCard\sDraw|(5|6)\sCard\sOmaha(\sHi/Lo)?|Courchevel(\sHi/Lo)?)\s
          (?P<LIMIT>No\sLimit|NO\sLIMIT|Fixed\sLimit|Limit|LIMIT|Pot\sLimit|POT\sLIMIT|Pot\sLimit\sPre\-Flop,\sNo\sLimit\sPost\-Flop)\)?,?\s
          (-\s)?
          (?P<SHOOTOUT>Match.*,\s)?
          ((Level|LEVEL)\s(?P<LEVEL>[IVXLC\d]+)\s)?
          \(?                            # open paren of the stakes
          (?P<CURRENCY>%(LS)s|)?
          (ante\s\d+,\s)?
          ((?P<SB>[.0-9]+)/(%(LS)s)?(?P<BB>[.0-9]+)|Button\sBlind\s(?P<CURRENCY1>%(LS)s|)(?P<BUB>[.0-9]+)\s\-\sAnte\s(%(LS)s)?[.0-9]+\s)
          (?P<CAP>\s-\s[%(LS)s]?(?P<CAPAMT>[.0-9]+)\sCap\s-\s)?        # Optional Cap part
          \s?(?P<ISO>%(LEGAL_ISO)s)?
          \)                        # close paren of the stakes
          (?P<BLAH2>\s\[AAMS\sID:\s[A-Z0-9]+\])?         # AAMS ID: in .it HH's
          \s-\s
          (?P<DATETIME>.*$)  """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_GameInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SITE,TITLE,HID,TOUR,TOURNO,HIVETABLE,BUYIN,BIAMT,BIRAKE,BOUNTY,TOUR_ISO,MIXED,SPLIT,GAME,LIMIT,SHOOTOUT,LEVEL,CURRENCY,SB,BB,CURRENCY1,BUB,CAP,CAPAMT,ISO,BLAH2,DATETIME
0,PokerStars,Hand,242207161368,None,None,None,None,None,None,None,None,None,None,Fusion,Pot Limit,None,None,€,0.01,0.02,None,None,None,None,EUR,None,2023/03/03 20:38:34 CET [2023/03/03 14:38:34 ET]


In [12]:
re_PlayerInfo = re.compile(u"""\s?Seat\s(?P<SEAT>[0-9]+):\s
          (?P<PNAME>.*)\s
          \((%(LS)s)?(?P<CASH>[,.0-9]+)\sin\schips
          (,\s(%(LS)s)?(?P<BOUNTY>[,.0-9]+)\sbounty)?
          \)
          (?P<SITOUT>\sis\ssitting\sout)?  """ %  substitutions, re.MULTILINE|re.VERBOSE)


if m != None:
    
    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))



,SITE,TITLE,HID,TOUR,TOURNO,HIVETABLE,BUYIN,BIAMT,BIRAKE,BOUNTY,TOUR_ISO,MIXED,SPLIT,GAME,LIMIT,SHOOTOUT,LEVEL,CURRENCY,SB,BB,CURRENCY1,BUB,CAP,CAPAMT,ISO,BLAH2,DATETIME
0,PokerStars,Hand,242207161368,None,None,None,None,None,None,None,None,None,None,Fusion,Pot Limit,None,None,€,0.01,0.02,None,None,None,None,EUR,None,2023/03/03 20:38:34 CET [2023/03/03 14:38:34 ET]


In [13]:
re_HandInfo = re.compile(u"""\s?Table\s(ID\s)?\'(?P<TABLE>.+?)\'\s
          ((?P<MAX>\d+)-[Mm]ax\s)?
          (?P<PLAY>\(Play\sMoney\)\s)?
          (Seat\s\#(?P<BUTTON>\d+)\sis\sthe\sbutton)?  """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_HandInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,TABLE,MAX,PLAY,BUTTON
0,Baetsle III,6,None,3


In [14]:
re_Identify = re.compile(u"""(PokerStars|POKERSTARS|Hive\sPoker|Full\sTilt|PokerMaster|Run\sIt\sOnce\sPoker|BetOnline|PokerBros)(\sGame|\sHand|\sHome\sGame|\sHome\sGame\sHand|Game|\s(Zoom|Rush)\sHand|\sGAME)\s\#\d+: """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_GameInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SITE,TITLE,HID,TOUR,TOURNO,HIVETABLE,BUYIN,BIAMT,BIRAKE,BOUNTY,TOUR_ISO,MIXED,SPLIT,GAME,LIMIT,SHOOTOUT,LEVEL,CURRENCY,SB,BB,CURRENCY1,BUB,CAP,CAPAMT,ISO,BLAH2,DATETIME
0,PokerStars,Hand,242207161368,None,None,None,None,None,None,None,None,None,None,Fusion,Pot Limit,None,None,€,0.01,0.02,None,None,None,None,EUR,None,2023/03/03 20:38:34 CET [2023/03/03 14:38:34 ET]


In [15]:
re_Button = re.compile(u"""Seat\s\#(?P<BUTTON>\d+)\sis\sthe\sbutton
                        """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Button.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,BUTTON
0,3


In [16]:
re_Board = re.compile(u"""\[(?P<CARDS>.+)\]
                        """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Board.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,CARDS
0,2023/03/03 14:38:34 ET


In [17]:
re_Board2       = re.compile(r"\[(?P<C1>\S\S)\] \[(\S\S)?(?P<C2>\S\S) (?P<C3>\S\S)\]")

m = re_Board2.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [18]:
re_DateTime1 = re.compile(u"""(?P<Y>[0-9]{4})\/(?P<M>[0-9]{2})\/(?P<D>[0-9]{2})[\- ]+(?P<H>[0-9]+):(?P<MIN>[0-9]+):(?P<S>[0-9]+)
                        """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_DateTime1.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,Y,M,D,H,MIN,S
0,2023,03,03,20,38,34


In [19]:
re_DateTime2 = re.compile(u"""(?P<Y>[0-9]{4})\/(?P<M>[0-9]{2})\/(?P<D>[0-9]{2})[\- ]+(?P<H>[0-9]+):(?P<MIN>[0-9]+)
                        """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_DateTime2.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,Y,M,D,H,MIN
0,2023,03,03,20,38


In [20]:
re_Action = re.compile(u"""%(PLYR)s:(?P<ATYPE>\sbets|\schecks|\sraises|\scalls|\sfolds|\sdiscards|\sstands\spat)
                        (\s%(CUR)s(?P<BET>[,.\d]+))?(\sto\s%(CUR)s(?P<BETTO>[,.\d]+))?  # the number discarded goes in <BET>
                        \s*(and\sis\sall.in)?
                        (and\shas\sreached\sthe\s[%(CUR)s\d\.,]+\scap)?
                        (\son|\scards?)?
                        (\s\(disconnect\))?
                        (\s\[(?P<CARDS>.+?)\])?\s*$
                        """ %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Action.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,PNAME,ATYPE,BET,BETTO,CARDS
0,MERGUEZ-3510,folds,None,None,None


In [24]:
re_ShowdownAction = re.compile(u"""%s: (shows|mucks|mucked|showed) \[(?P<CARDS>.*)\]""" % substitutions['PLYR'], re.MULTILINE)
#m = re_ShowdownAction.search(test)
for shows in re_ShowdownAction.finditer(test):            
    
    
    if shows != None:

        mg = shows.groupdict()

        df = pd.DataFrame.from_dict(mg, orient='index')
        df = df.transpose()
        display(HTML(df.to_html()))

,PNAME,CARDS
0,joconde421,Qc 8h Ah 6s


In [25]:
re_CollectPot = re.compile(u"Seat (?P<SEAT>[0-9]+): %(PLYR)s %(BRKTS)s(collected|showed \[.*\] and (won|collected)) \(?%(CUR)s(?P<POT>[,.\d]+)\)?(, mucked| with.*|)" %  substitutions, re.MULTILINE)
m = re_CollectPot.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SEAT,PNAME,POT
0,5,joconde421,0.04


In [26]:
re_CollectPot2      = re.compile(r"%(PLYR)s (collected|cashed out the hand for) %(CUR)s(?P<POT>[,.\d]+)" %  substitutions, re.MULTILINE)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SEAT,PNAME,POT
0,5,joconde421,0.04


In [27]:
re_CashedOut = re.compile(r"cashed\sout\sthe\shand")

m = re_CashedOut.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [28]:
re_WinningRankOne = re.compile(u"%(PLYR)s wins the tournament and receives %(CUR)s(?P<AMT>[,\.0-9]+) - congratulations!$" %  substitutions, re.MULTILINE)

m = re_WinningRankOne.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [29]:
re_WinningRankOther = re.compile(u"%(PLYR)s finished the tournament in (?P<RANK>[0-9]+)(st|nd|rd|th) place and received %(CUR)s(?P<AMT>[,.0-9]+)\.$" %  substitutions, re.MULTILINE)

m = re_WinningRankOther.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [30]:
re_Uncalled   = re.compile('Uncalled bet \(%(CUR)s(?P<BET>[,.\d]+)\) returned to' %  substitutions, re.MULTILINE)

m = re_Uncalled.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [31]:
re_Rake             = re.compile(u"""
                        Total\spot\s%(CUR)s(?P<POT>[,\.0-9]+)(.+?)?\s\|\sRake\s%(CUR)s(?P<RAKE>[,\.0-9]+)"""
                         %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Rake.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,POT,RAKE
0,0.04,0


In [32]:
re_STP             = re.compile(u"""
                        STP\sadded:\s%(CUR)s(?P<AMOUNT>[,\.0-9]+)"""
                         %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_STP.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

In [33]:
player_re = "jeje_sat"
subst = {
                'PLYR': player_re,
                'BRKTS': r'(\(button\) |\(small blind\) |\(big blind\) |\(button\) \(small blind\) |\(button\) \(big blind\) )?',
                'CUR': u"(\$|\xe2\x82\xac|\u20ac||\£|)"
            }
re_HeroCards = re.compile(r"Dealt to %(PLYR)s(?: \[(?P<OLDCARDS>.+?)\])?( \[(?P<NEWCARDS>.+?)\])" % subst, re.MULTILINE)



m = re_HeroCards.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,OLDCARDS,NEWCARDS
0,None,6h Jh


In [22]:
m =  re.search(r"\*\*\* HOLE CARDS \*\*\*(?P<PREFLOP>(.+(?P<FLOPET>\[\S\S\]))?.+(?=\*\*\* (FIRST\s)?FLOP \*\*\*)|.+)"
                           r"(\*\*\* FLOP \*\*\*(?P<FLOP> (\[\S\S\] )?\[(\S\S ?)?\S\S \S\S\].+(?=\*\*\* (FIRST\s)?TURN \*\*\*)|.+))?"
                           r"(\*\*\* TURN \*\*\* \[\S\S \S\S \S\S] (?P<TURN>\[\S\S\].+(?=\*\*\* (FIRST\s)?RIVER \*\*\*)|.+))?"
                           r"(\*\*\* RIVER \*\*\* \[\S\S \S\S \S\S \S\S] (?P<RIVER>\[\S\S\].+))?"
                           r"(\*\*\* FIRST FLOP \*\*\*(?P<FLOP1> (\[\S\S\] )?\[(\S\S ?)?\S\S \S\S\].+(?=\*\*\* FIRST TURN \*\*\*)|.+))?"
                           r"(\*\*\* FIRST TURN \*\*\* \[\S\S \S\S \S\S] (?P<TURN1>\[\S\S\].+(?=\*\*\* FIRST RIVER \*\*\*)|.+))?"
                           r"(\*\*\* FIRST RIVER \*\*\* \[\S\S \S\S \S\S \S\S] (?P<RIVER1>\[\S\S\].+?(?=\*\*\* SECOND (FLOP|TURN|RIVER) \*\*\*)|.+))?"
                           r"(\*\*\* SECOND FLOP \*\*\*(?P<FLOP2> (\[\S\S\] )?\[\S\S ?\S\S \S\S\].+(?=\*\*\* SECOND TURN \*\*\*)|.+))?"
                           r"(\*\*\* SECOND TURN \*\*\* \[\S\S \S\S \S\S] (?P<TURN2>\[\S\S\].+(?=\*\*\* SECOND RIVER \*\*\*)|.+))?"
                           r"(\*\*\* SECOND RIVER \*\*\* \[\S\S \S\S \S\S \S\S] (?P<RIVER2>\[\S\S\].+))?", test,re.DOTALL)

if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))
    

,PREFLOP,FLOPET,FLOP,TURN,RIVER,FLOP1,TURN1,RIVER1,FLOP2,TURN2,RIVER2
0,\nDealt to jeje_sat [Th Tc]\nCash8201: calls €0.02\njeje_sat: raises €0.07 to €0.09\nhnaolih: calls €0.09\ncapitainefla: folds \nKazdal95: calls €0.07\nCash8201: calls €0.07\n,None,[Jd 3h 4d]\nDealt to jeje_sat [Th Tc] [4s]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n,[Td]\nDealt to jeje_sat [Th Tc 4s] [4c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n,"[6c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: bets €0.31 and is all-in\nKazdal95: folds \nCash8201: calls €0.31\njeje_sat: folds \n*** SHOW DOWN ***\nhnaolih: shows [5d Qs Ah 7d] (a flush, Jack high)\nCash8201: shows [Jc 7h Ac 5c] (a straight, Three to Seven)\nhnaolih collected €0.93 from pot\n*** SUMMARY ***\nTotal pot €0.99 | Rake €0.06 \nBoard [Jd 3h 4d Td 6c]\nSeat 2: Cash8201 showed [Jc 7h Ac 5c] and lost with a straight, Three to Seven\nSeat 3: jeje_sat folded on the River\nSeat 4: hnaolih (button) showed [5d Qs Ah 7d] and won (€0.93) with a flush, Jack high\nSeat 5: capitainefla (small blind) folded before Flop\nSeat 6: Kazdal95 (big blind) folded on the River",None,None,None,None,None,None


In [10]:
mm = {'PREFLOP': '\nDealt to jeje_sat [Th Tc]\nCash8201: calls €0.02\njeje_sat: raises €0.07 to €0.09\nhnaolih: calls €0.09\ncapitainefla: folds \nKazdal95: calls €0.07\nCash8201: calls €0.07\n', 'FLOPET': None, 'FLOP': ' [Jd 3h 4d]\nDealt to jeje_sat [Th Tc] [4s]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n', 'TURN': '[Td]\nDealt to jeje_sat [Th Tc 4s] [4c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n', 'RIVER': '[6c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: bets €0.31 and is all-in\nKazdal95: folds \nCash8201: calls €0.31\njeje_sat: folds \n*** SHOW DOWN ***\nhnaolih: shows [5d Qs Ah 7d] (a flush, Jack high)\nCash8201: shows [Jc 7h Ac 5c] (a straight, Three to Seven)\nhnaolih collected €0.93 from pot\n*** SUMMARY ***\nTotal pot €0.99 | Rake €0.06 \nBoard [Jd 3h 4d Td 6c]\nSeat 2: Cash8201 showed [Jc 7h Ac 5c] and lost with a straight, Three to Seven\nSeat 3: jeje_sat folded on the River\nSeat 4: hnaolih (button) showed [5d Qs Ah 7d] and won (€0.93) with a flush, Jack high\nSeat 5: capitainefla (small blind) folded before Flop\nSeat 6: Kazdal95 (big blind) folded on the River', 'FLOP1': None, 'TURN1': None, 'RIVER1': None, 'FLOP2': None, 'TURN2': None, 'RIVER2': None}
m = {'PREFLOP': '', 'FLOP':'', 'TURN':'', 'RIVER': ''}
m.update(mm)
print(m)


{'PREFLOP': '\nDealt to jeje_sat [Th Tc]\nCash8201: calls €0.02\njeje_sat: raises €0.07 to €0.09\nhnaolih: calls €0.09\ncapitainefla: folds \nKazdal95: calls €0.07\nCash8201: calls €0.07\n', 'FLOP': ' [Jd 3h 4d]\nDealt to jeje_sat [Th Tc] [4s]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n', 'TURN': '[Td]\nDealt to jeje_sat [Th Tc 4s] [4c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: checks \n', 'RIVER': '[6c]\nKazdal95: checks \nCash8201: checks \njeje_sat: checks \nhnaolih: bets €0.31 and is all-in\nKazdal95: folds \nCash8201: calls €0.31\njeje_sat: folds \n*** SHOW DOWN ***\nhnaolih: shows [5d Qs Ah 7d] (a flush, Jack high)\nCash8201: shows [Jc 7h Ac 5c] (a straight, Three to Seven)\nhnaolih collected €0.93 from pot\n*** SUMMARY ***\nTotal pot €0.99 | Rake €0.06 \nBoard [Jd 3h 4d Td 6c]\nSeat 2: Cash8201 showed [Jc 7h Ac 5c] and lost with a straight, Three to Seven\nSeat 3: jeje_sat folded on the River\nSeat 4: hnaolih (button) sho